In [9]:
import sqlite3
import re

In [10]:
# Connect to the database (creates the database file if it doesn't exist)
conn = sqlite3.connect('email_counts.db')
cur = conn.cursor()

In [11]:
# Drop the table if it exists (to clear data before each run)
cur.execute('DROP TABLE IF EXISTS Counts')

In [12]:
# Create the table
cur.execute('CREATE TABLE Counts (org TEXT, count INTEGER)')

In [13]:
# Read the mbox.txt file and update the database
filename = input("Enter filename: ")
try:
    with open(filename, 'r', encoding='utf-8') as file:
        for line in file:
            if line.startswith('From: '):
                email = line.split()[1]
                domain = re.findall('@([^ ]+)', email)
                if domain:
                    domain = domain[0]
                    cur.execute('SELECT count FROM Counts WHERE org = ?', (domain,))
                    row = cur.fetchone()
                    if row is None:
                        cur.execute('INSERT INTO Counts (org, count) VALUES (?, 1)', (domain,))
                    else:
                        cur.execute('UPDATE Counts SET count = count + 1 WHERE org = ?', (domain,))
        conn.commit()
except Exception as e:
    print(f"An error occurred: {e}")

Enter filename: mbox.txt


In [14]:
# Print the results
cur.execute('SELECT * FROM Counts ORDER BY count DESC')
for row in cur.fetchall():
    print(row)

('iupui.edu', 8)
('umich.edu', 7)
('uct.ac.za', 6)
('media.berkeley.edu', 4)
('caret.cam.ac.uk', 1)
('gmail.com', 1)


In [15]:
# Close the database connection
cur.close()